In [0]:
filename = "data_hindi.txt"
data = open(filename, 'r', encoding='utf-8').read()
print(data)

"ये लंदन है, दुनिया का सबसे बड़ा शहर।"
"मैं 22 वर्ष से यहाँ रह रहा हूँ, रोज़ाना इसी सड़क से गुजरता हूँ और रोज़ाना ये सड़क मुझसे मेरा नाम पूछती है।"
ये पूछती है कि चौधरी बलदेव सिंह कौन है? कहां से आया है? क्यों आया है?
"अब मैं इसे क्या जवाब दूँ, आधी उम्र गुज़ारने के बाद भी ये धरती मेरे लिए अजनबी है और मैं इसके लिए।"
"मॉम! बीस वर्ष से डैड एक ही जगह जाते हैं, एक ही रास्ते से जाते हैं, फिर भी आप रोज फोन करके पूछती हैं, ‘तुसी पहुँच गये जी?’"
"तू नहीं समझ पायेगी, बस इतना समझ ले अब आदत सी पड़ गयी है।"
मिस लूसी सही कहती है आदतें अगर वक्त पर ना बदली जाएं तो जरूरतें बन जाती हैं।
"फिर झाड़ी फिलॉसफी, अरे तू बारह बरस की है, बारह बरस वालों जैसी बात किया कर ना, अपनी उम्र से बड़ी-बड़ी बातें क्यूँ करती है तू?"
"उफ मॉम! बचपन जवानी कोई चीज नहीं होती, इंसान की उम्र उतनी ही होती है, जितनी वो फील करता है।"
"अच्छा बाबा मैं हार गयी, मैं हार गयी।"
अब तू अपने स्कूल जा और अपनी मिस लूसी का सिर खा।
"ओह! मुझसे भला क्या परदा, देख जब लड़की जवान हो जाती है ना तो माँ उसकी माँ नहीं, सहेली बन जाती है।"
"अरे सिमी! तू तो बड़ी अच्छ

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Bidirectional

from keras.layers import Embedding

Using TensorFlow backend.


In [0]:
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
	in_text = seed_text
    # generate a fixed number of words
	for _ in range(n_words):
        # encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# pre-pad sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
	return in_text

In [0]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]

In [0]:
# retrieve vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# encode 2 words -> 1 word
sequences = list()
for i in range(2, len(encoded)):
	sequence = encoded[i-2:i+1]
	sequences.append(sequence)


Vocabulary Size: 2573


In [0]:
print('Total Sequences: %d' % len(sequences))
# pad sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)

Total Sequences: 14522
Max Sequence Length: 3


In [0]:
# split into input and output elements
from numpy import array

sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]

#print(X)
y = to_categorical(y, num_classes=vocab_size)
#print(y)
# define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_length-1))
model.add(Bidirectional(LSTM(50)))
model.add(Dense(vocab_size, activation='softmax'))

In [0]:
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(X, y, epochs=500, verbose=2)
# evaluate model

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2, 10)             25730     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100)               24400     
_________________________________________________________________
dense_1 (Dense)              (None, 2573)              259873    
Total params: 310,003
Trainable params: 310,003
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/500
 - 5s - loss: 6.9369 - accuracy: 0.0275
Epoch 2/500
 - 4s - loss: 6.4835 - accuracy: 0.0281
Epoch 3/500
 - 4s - loss: 6.3949 - accuracy: 0.0298
Epoch 4/500
 - 4s - loss: 6.3049 - accuracy: 0.0353
Epoch 5/500
 - 4s - loss: 6.2029 - accuracy: 0.0403
Epoch 6/500
 - 4s - loss: 6.1149 - accuracy: 0.0432
Epoch 7/500
 - 4s - loss: 6.0322 - accuracy: 0.0468
Epoch 8/500
 - 4s - loss: 5.9538 - accuracy: 0.0494
Epoch 9/500
 - 4s - loss: 5.8767 - accuracy: 0.0526
Epoch 10/500
 - 4s - loss: 5.7972 - accuracy: 0.0554
Epoch 11/500
 - 4s - loss: 5.7105 - accuracy: 0.0578
Epoch 12/500
 - 4s - loss: 5.6145 - accuracy: 0.0640
Epoch 13/500
 - 4s - loss: 5.5134 - accuracy: 0.0702
Epoch 14/500
 - 4s - loss: 5.4059 - accuracy: 0.0760
Epoch 15/500
 - 4s - loss: 5.2927 - accuracy: 0.0830
Epoch 16/500
 - 4s - loss: 5.1757 - accuracy: 0.0915
Epoch 17/500
 - 4s - loss: 5.0563 - accuracy: 0.0989
Epoch 18/500
 - 4s - loss: 4.9350 - accuracy: 0.1091
Epoch 19/500
 - 4s - loss: 4.8155 - accuracy: 0.1193
Ep

In [0]:
print(generate_seq(model, tokenizer, max_length-1, 'सितारे दिन में', 2))
print(generate_seq(model, tokenizer, max_length-1, 'खुशी', 2))
print(generate_seq(model, tokenizer, max_length-1, 'आपसे मिलकर', 2))
